In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN2_256/'

In [4]:
pretrained = ''
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 200
start_epoch = 0
train_batch = 190
test_batch = 190
lr = 0.015
schedule = [20, 75, 125, 175]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style2/128/b1_2' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')    
train_aug = transforms.Compose([
#     transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.3, 'drop_connect_rate':0.3})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [9]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 6.52M


In [10]:
# summary(model, input_size=(3,64,64), device='cuda')

# Loss

In [11]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4, nesterov=True)
# optimizer = optim.Adam(model.parameters(), weight_decay=1e-4)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [12]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

# Train

In [13]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [14]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [15]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [16]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    scheduler_warmup.step()

    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [1 | 200] LR: 0.015000
1/408 Data:12.778 | Batch:15.000 | Total:0:00:15 | ETA:1:41:46 | Loss:0.7379807233810425 | top1:44.73684310913086
11/408 Data:0.000 | Batch:0.310 | Total:0:00:32 | ETA:0:21:24 | Loss:0.7040432637388055 | top1:52.3923454284668
21/408 Data:12.272 | Batch:12.619 | Total:0:01:09 | ETA:0:23:59 | Loss:0.6989002256166368 | top1:53.38345718383789
31/408 Data:0.000 | Batch:0.311 | Total:0:01:36 | ETA:0:24:26 | Loss:0.7114043081960371 | top1:52.954158782958984
41/408 Data:9.402 | Batch:9.725 | Total:0:02:15 | ETA:0:23:44 | Loss:0.7074199217121776 | top1:53.183570861816406
51/408 Data:3.784 | Batch:4.123 | Total:0:02:41 | ETA:0:15:38 | Loss:0.7062285110062244 | top1:53.06501770019531
61/408 Data:7.721 | Batch:8.061 | Total:0:03:12 | ETA:0:18:11 | Loss:0.7049562403413115 | top1:52.88179397583008
71/408 Data:0.017 | Batch:0.337 | Total:0:03:41 | ETA:0:20:30 | Loss:0.7054958184000472 | top1:52.51297378540039
81/408 Data:10.163 | Batch:10.507 | Total:0:04:17 | ETA:0:19:

/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


42/42 Data:0.000 | Batch:0.520 | Total:0:00:22 | ETA:0:00:00 | Loss:0.7145749143300912 | top1:50.0

Epoch: [2 | 200] LR: 0.025500
1/408 Data:1.114 | Batch:1.789 | Total:0:00:01 | ETA:0:12:09 | Loss:0.7156342267990112 | top1:53.157901763916016
11/408 Data:0.000 | Batch:0.319 | Total:0:00:07 | ETA:0:04:39 | Loss:0.6983719143000516 | top1:52.20095443725586
21/408 Data:0.001 | Batch:0.655 | Total:0:00:13 | ETA:0:03:53 | Loss:0.7021206929570153 | top1:51.50375747680664
31/408 Data:0.000 | Batch:0.662 | Total:0:00:19 | ETA:0:04:11 | Loss:0.7053514795918618 | top1:50.74702835083008
41/408 Data:0.001 | Batch:0.665 | Total:0:00:26 | ETA:0:04:05 | Loss:0.7018425726309055 | top1:51.5917854309082
51/408 Data:0.001 | Batch:0.664 | Total:0:00:33 | ETA:0:03:57 | Loss:0.700778975206263 | top1:51.66150665283203
61/408 Data:0.001 | Batch:0.663 | Total:0:00:39 | ETA:0:03:51 | Loss:0.7001529257805621 | top1:51.760135650634766
71/408 Data:0.001 | Batch:0.663 | Total:0:00:46 | ETA:0:03:44 | Loss:0.699437584

291/408 Data:0.001 | Batch:0.663 | Total:0:03:18 | ETA:0:01:18 | Loss:0.692133186404238 | top1:53.47621536254883
301/408 Data:0.001 | Batch:0.574 | Total:0:03:24 | ETA:0:01:12 | Loss:0.6919730133392487 | top1:53.540828704833984
311/408 Data:0.001 | Batch:0.623 | Total:0:03:29 | ETA:0:00:52 | Loss:0.691877645886596 | top1:53.567440032958984
321/408 Data:0.000 | Batch:0.515 | Total:0:03:34 | ETA:0:00:48 | Loss:0.6919849123167472 | top1:53.536643981933594
331/408 Data:0.001 | Batch:0.658 | Total:0:03:41 | ETA:0:00:50 | Loss:0.6919667821636373 | top1:53.52361297607422
341/408 Data:0.001 | Batch:0.665 | Total:0:03:48 | ETA:0:00:45 | Loss:0.6917544147136274 | top1:53.597774505615234
351/408 Data:0.001 | Batch:0.669 | Total:0:03:54 | ETA:0:00:38 | Loss:0.6917238440948335 | top1:53.648223876953125
361/408 Data:0.001 | Batch:0.664 | Total:0:04:01 | ETA:0:00:32 | Loss:0.691688907938981 | top1:53.659423828125
371/408 Data:0.001 | Batch:0.668 | Total:0:04:08 | ETA:0:00:25 | Loss:0.6916365215399195

171/408 Data:0.001 | Batch:0.666 | Total:0:01:54 | ETA:0:02:38 | Loss:0.5685811117727156 | top1:70.67097473144531
181/408 Data:0.001 | Batch:0.664 | Total:0:02:01 | ETA:0:02:31 | Loss:0.5637165665297218 | top1:71.03518676757812
191/408 Data:0.002 | Batch:0.677 | Total:0:02:08 | ETA:0:02:25 | Loss:0.5599573510479553 | top1:71.33370208740234
201/408 Data:0.011 | Batch:0.700 | Total:0:02:14 | ETA:0:02:21 | Loss:0.5570534713825777 | top1:71.5108642578125
211/408 Data:0.001 | Batch:0.497 | Total:0:02:20 | ETA:0:02:05 | Loss:0.5532492283396246 | top1:71.758544921875
221/408 Data:0.006 | Batch:0.398 | Total:0:02:25 | ETA:0:01:33 | Loss:0.5498412568374996 | top1:72.01000213623047
231/408 Data:0.001 | Batch:0.582 | Total:0:02:30 | ETA:0:01:24 | Loss:0.5459831311847224 | top1:72.3171615600586
241/408 Data:0.001 | Batch:0.453 | Total:0:02:35 | ETA:0:01:30 | Loss:0.542763236770986 | top1:72.57042694091797
251/408 Data:0.001 | Batch:0.449 | Total:0:02:40 | ETA:0:01:16 | Loss:0.5394665431453887 | to

...

351/408 Data:0.002 | Batch:0.659 | Total:0:03:45 | ETA:0:00:38 | Loss:0.0015041678231301554 | top1:99.9460220336914
361/408 Data:0.002 | Batch:0.668 | Total:0:03:51 | ETA:0:00:32 | Loss:0.001516049813698149 | top1:99.94459533691406
371/408 Data:0.001 | Batch:0.586 | Total:0:03:58 | ETA:0:00:25 | Loss:0.0015821031850873745 | top1:99.94325256347656
381/408 Data:0.001 | Batch:0.666 | Total:0:04:04 | ETA:0:00:18 | Loss:0.0015565478393131668 | top1:99.94474792480469
391/408 Data:0.002 | Batch:0.649 | Total:0:04:11 | ETA:0:00:12 | Loss:0.001586933123699264 | top1:99.9421157836914
401/408 Data:0.002 | Batch:0.716 | Total:0:04:18 | ETA:0:00:05 | Loss:0.001581260694442597 | top1:99.94224548339844
42/42 Data:0.003 | Batch:0.075 | Total:0:00:14 | ETA:0:00:00 | Loss:0.010648132662588931 | top1:99.75640869140625

Epoch: [107 | 200] LR: 0.000647
1/408 Data:1.833 | Batch:2.483 | Total:0:00:02 | ETA:0:16:51 | Loss:0.00010851307888515294 | top1:100.00000762939453
11/408 Data:0.002 | Batch:0.631 | Total

221/408 Data:0.021 | Batch:0.726 | Total:0:02:23 | ETA:0:02:16 | Loss:0.001857161177291321 | top1:99.94284057617188
231/408 Data:0.002 | Batch:0.646 | Total:0:02:30 | ETA:0:02:00 | Loss:0.001784840368407348 | top1:99.94532012939453
241/408 Data:0.003 | Batch:0.668 | Total:0:02:37 | ETA:0:02:01 | Loss:0.0017186939276219153 | top1:99.94758605957031
251/408 Data:0.002 | Batch:0.719 | Total:0:02:44 | ETA:0:01:45 | Loss:0.0017199911765699261 | top1:99.94757843017578
261/408 Data:0.001 | Batch:0.571 | Total:0:02:51 | ETA:0:01:43 | Loss:0.0017161022866463915 | top1:99.945556640625
271/408 Data:0.013 | Batch:0.686 | Total:0:02:57 | ETA:0:01:31 | Loss:0.0017264129281008244 | top1:99.94367218017578
281/408 Data:0.006 | Batch:0.722 | Total:0:03:05 | ETA:0:01:32 | Loss:0.0016700323185594185 | top1:99.94568634033203
291/408 Data:0.006 | Batch:0.691 | Total:0:03:11 | ETA:0:01:16 | Loss:0.0016200515626608358 | top1:99.94754791259766
301/408 Data:0.003 | Batch:0.666 | Total:0:03:18 | ETA:0:01:14 | Los

81/408 Data:0.002 | Batch:0.650 | Total:0:00:58 | ETA:0:03:40 | Loss:0.0009495729571449093 | top1:99.9675064086914
91/408 Data:0.002 | Batch:0.670 | Total:0:01:05 | ETA:0:03:33 | Loss:0.0014918618188268418 | top1:99.94794464111328
101/408 Data:0.001 | Batch:0.692 | Total:0:01:11 | ETA:0:03:24 | Loss:0.0013750147705236082 | top1:99.9531021118164
111/408 Data:0.002 | Batch:0.683 | Total:0:01:18 | ETA:0:03:21 | Loss:0.001296287085350995 | top1:99.95732879638672
121/408 Data:0.001 | Batch:0.682 | Total:0:01:25 | ETA:0:03:12 | Loss:0.0013218993509394187 | top1:99.95650482177734
131/408 Data:0.002 | Batch:0.670 | Total:0:01:32 | ETA:0:03:08 | Loss:0.00154621876886302 | top1:99.9477767944336
141/408 Data:0.013 | Batch:0.688 | Total:0:01:38 | ETA:0:03:00 | Loss:0.0016509728177602516 | top1:99.94400787353516
151/408 Data:0.001 | Batch:0.669 | Total:0:01:45 | ETA:0:02:54 | Loss:0.0015835803195004218 | top1:99.94772338867188
161/408 Data:0.001 | Batch:0.672 | Total:0:01:52 | ETA:0:02:46 | Loss:0.

371/408 Data:0.000 | Batch:0.679 | Total:0:03:57 | ETA:0:00:25 | Loss:0.0018000773859359047 | top1:99.94467163085938
381/408 Data:0.007 | Batch:0.671 | Total:0:04:04 | ETA:0:00:19 | Loss:0.0018217432247508401 | top1:99.94336700439453
391/408 Data:0.001 | Batch:0.700 | Total:0:04:11 | ETA:0:00:12 | Loss:0.0017886523915769752 | top1:99.94480895996094
401/408 Data:0.001 | Batch:0.671 | Total:0:04:18 | ETA:0:00:05 | Loss:0.0017775719790457298 | top1:99.94486999511719
42/42 Data:0.000 | Batch:0.057 | Total:0:00:15 | ETA:0:00:00 | Loss:0.010636603072650056 | top1:99.74358367919922

Epoch: [112 | 200] LR: 0.000600
1/408 Data:2.149 | Batch:2.601 | Total:0:00:02 | ETA:0:17:39 | Loss:0.00013260464766062796 | top1:100.00000762939453
11/408 Data:0.001 | Batch:0.578 | Total:0:00:07 | ETA:0:04:34 | Loss:0.001010034435735592 | top1:99.9521484375
21/408 Data:0.000 | Batch:0.429 | Total:0:00:12 | ETA:0:03:14 | Loss:0.0006322810906914085 | top1:99.97493743896484
31/408 Data:0.000 | Batch:0.666 | Total:0

KeyboardInterrupt: 